# **QA Chatbot**

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain import SerpAPIWrapper
from langchain.utilities import SerpAPIWrapper
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain import LLMMathChain
import os
import config

In [8]:
# %pip install -r requirements.txt

In [3]:
API_KEY = os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

In [4]:
# Definir constantes
CHROMA_PATH = "chroma"  # Ruta para la base de datos de Chroma
DATA_PATH = "./"  # Ruta al directorio que contiene los datos

In [5]:
file_pattern = 'embeddings.md'

## **Embeddings**

In [6]:
# Instanciación del cargador de directorio con el patrón de archivo especificado
loader = DirectoryLoader(DATA_PATH, glob=file_pattern,
                         loader_cls=lambda path: TextLoader(path, encoding='utf-8'))

# Carga de documentos utilizando el cargador
documents = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

141

In [9]:
texts[0]

Document(page_content='# Plan de Estudios: Ingeniería en Datos e Inteligencia Organizacional (Plan 2016)\n\n## Créditos Totales\n\n404 créditos\n\n## Descripción General', metadata={'source': 'embeddings.md'})

In [10]:
# Directorio de persistencia para almacenar la base de datos
persist_directory = 'db'

# Seleccionamos OpenAI embeddings para la generación de vectores
embedding = OpenAIEmbeddings()

# Creamos una instancia de Chroma para generar vectores a partir de documentos
vectordb = Chroma.from_documents(
    documents=texts,  # Los documentos de texto a procesar para la generación de vectores
    embedding=embedding,  # El método de embedding a utilizar para generar vectores
    persist_directory=persist_directory  # Directorio donde se almacenará la base de datos de vectores
)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
vectordb.persist()
vectordb = None

In [12]:
# Creación de una nueva instancia de Chroma, para cargar una base de datos existente
vectordb = Chroma(
    persist_directory=persist_directory,  # Directorio de persistencia de la base de datos de vectores
    embedding_function=embedding  # Función de embedding a utilizar
)

In [13]:
# Convertir la base de datos de vectores en un objeto retriever
retriever = vectordb.as_retriever()

In [14]:
# Usando el objeto retriever para obtener documentos relevantes para una consulta específica
docs = retriever.get_relevant_documents("Total de creditos de la carrera")

In [15]:
docs

[Document(page_content='404 créditos\n\n## Descripción General\n\nTodas las materias de la Ingeniería en Datos e Inteligencia Organizacional tienen un ciclo, una clave, un nombre, un número de créditos, un tipo y una academia.', metadata={'source': 'embeddings.md'}),
 Document(page_content='# Plan de Estudios: Ingeniería en Datos e Inteligencia Organizacional (Plan 2016)\n\n## Créditos Totales\n\n404 créditos\n\n## Descripción General', metadata={'source': 'embeddings.md'}),
 Document(page_content='el plan de estudios, los 2 talleres (uno artístico/cucural y uno deportivo), tópicos 2, nivel 6 del tercer idioma y el servicio social liberado. Es importante cubrir el total de créditos por sección.', metadata={'source': 'embeddings.md'}),
 Document(page_content='Durante el tercer ciclo debes cargar y aprobar todas las materias basicas.\n\nLas materias de Elección Libre se juntan en el ciclo 3 y 4, se deben reunir 48 creditos de las materias de Elección Libre.', metadata={'source': 'embeddi

In [16]:
len(docs)

4

In [17]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [18]:
retriever.search_type

'similarity'

In [19]:
retriever.search_kwargs

{'k': 2}

In [20]:
# Creación de un objeto RetrievalQA a partir de un tipo de cadena específico
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),  # Modelo de lenguaje a utilizar para la generación de respuestas
    # Tipo de cadena (podría ser una configuración específica para el QA)
    chain_type="stuff",
    retriever=retriever,  # Objeto retriever utilizado para recuperar documentos relevantes
    # Especifica si se deben devolver los documentos de origen junto con las respuestas
    return_source_documents=True
)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [21]:
def process_llm_response(llm_response):
    # Imprimir la respuesta generada por el modelo de lenguaje
    print(llm_response['result'])

    # Imprimir la sección de fuentes de la respuesta
    print('\n\nSources:')

    # Iterar sobre cada fuente en los documentos de origen asociados con la respuesta
    for source in llm_response["source_documents"]:
        # Imprimir la fuente de cada documento de origen
        print(source.metadata['source'])

### Pruebas

In [22]:
# Consulta específica
query = "Quiero cargar Electricidad y magnetismo, ¿qué debo hacer?"

# Obtener la respuesta del modelo de pregunta-respuesta
llm_response = qa_chain(query)

# Procesar la respuesta utilizando la función process_llm_response
process_llm_response(llm_response)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Debes haber aprobado primero Física Clásica antes de cargar Electricidad y Magnetismo, ya que se recomienda cargar y aprobar Física Clásica antes de esta materia.


Sources:
embeddings.md
embeddings.md


In [23]:
query = "Quiero  cargar Calculo Integral, que debo hacer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Debes aprobar primero y cargar "Cálculo Diferencial" antes de "Cálculo Integral".


Sources:
embeddings.md
embeddings.md


In [24]:
query = "cuantos ciclos son?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 4


Sources:
embeddings.md
embeddings.md


In [25]:
query = "cual es la clave de la materia Pensamiento crítico para ingeniería y de que ciclo es?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La clave de la materia Pensamiento crítico para ingeniería es ID0160 y pertenece al ciclo 1.


Sources:
embeddings.md
embeddings.md


In [26]:
query = "¿cuantas materias y cuales debo aprobar antes de cargar Aprendizaje Estadistico?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Antes de cargar Aprendizaje Estadistico, debes aprobar todas las materias basicas de Ciclo 2 y Ciclo 3, y además aprobar por lo menos 2 materias de Eleccion Libre en cada ciclo.


Sources:
embeddings.md
embeddings.md


In [27]:
query = "¿cuantas materias y cuales debo aprobar antes de cargar Ecuaciones Diferenciales?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Antes de cargar Ecuaciones Diferenciales, debes aprobar todas las materias básicas de los dos primeros ciclos y al menos dos materias de Elección Libre del primer ciclo. No es necesario aprobar materias básicas del tercer ciclo para cargar Ecuaciones Diferenciales.


Sources:
embeddings.md
embeddings.md


In [28]:
query = "cuantos son los creditos totales de la carrera?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 No puedo responder esta pregunta ya que la información proporcionada no menciona la duración de la carrera. 


Sources:
embeddings.md
embeddings.md


In [29]:
query = " Hablame de la materia de Aprendizaje Estadistico"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La materia de Aprendizaje Estadístico es una materia básica del ciclo 3 con un valor de 6 créditos. Su clave es ID0309 y su nombre completo es "Aprendizaje estadístico". Es una materia que se enfoca en enseñar técnicas y métodos estadísticos para el análisis de datos y la toma de decisiones. 


Sources:
embeddings.md
embeddings.md


In [30]:
query = " cual es la clave de esa materia?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 No puedo responder a tu pregunta ya que no se a qué materia te refieres. Los datos proporcionados no incluyen ninguna materia en particular. 


Sources:
embeddings.md
embeddings.md


## **Agente: Google Search Results**

In [31]:
llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [32]:
os.environ["SERPAPI_API_KEY"] = config.SERPAPI_API_KEY

In [33]:
search = SerpAPIWrapper()

In [34]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [35]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

## **Agente: LLM MathChain**

In [36]:
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [37]:
tools = [
    Tool(
        name="QA System",
        func=qa_chain.run,
        description="Útil cuando necesitas responder preguntas sobre El mapa curricular de IDeIO. La entrada debe ser una pregunta completamente formulada."
    ),

    Tool(
        name="Backup QA Google Search",
        func=search.run,
        description="Útil cuando necesitas responder preguntas sobre El mapa curricular de IDeIO, pero solo cuando el Sistema de Preguntas y Respuestas de El mapa curricular de IDeIO no pudo responder la consulta. La entrada debe ser una pregunta completamente formulada."
    ),
    Tool(
        func=llm_math_chain.run,
        name="Calculadora",
        description="Herramienta que suma las materias y suma los creditos de las materias ",
    ),
]

In [38]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, prompt=prompt)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [39]:
agent.run("Cuantos creditos son en total?")

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I need to calculate the total number of credits in the curriculum.
Action: Calculadora
Action Input: Sumar los créditos de todas las materias

> Entering new LLMMathChain chain...
Sumar los créditos de todas las materiasLo siento, necesito una expresión matemática específica para poder traducirla a una expresión ejecutable en Python usando la biblioteca numexpr. ¿Podrías proporcionar la expresión matemática que deseas que calcule?

ValueError: unknown format from LLM: Lo siento, necesito una expresión matemática específica para poder traducirla a una expresión ejecutable en Python usando la biblioteca numexpr. ¿Podrías proporcionar la expresión matemática que deseas que calcule?

## **Process LLM**

In [ ]:
def process_llm_response(llm_response):
    # Imprimir la respuesta generada por el modelo de lenguaje
    print(llm_response['result'])

    # Imprimir la sección de fuentes de la respuesta
    print('\n\nSources:')

    # Iterar sobre cada fuente en los documentos de origen asociados con la respuesta
    for source in llm_response["source_documents"]:
        # Imprimir la fuente de cada documento de origen
        print(source.metadata['source'])

In [ ]:
query = "Quiero  cargar Calculo Integral, que debo hacer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Debes buscar la materia con clave II0209 en el ciclo 1 y asegurarte de que tenga 6 créditos y sea de tipo básica. Luego, debes inscribirte en ella para poder cursarla.


Sources:
embeddings.md
embeddings.md


In [ ]:
query = "cuantos ciclos son?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 
Hay 4 ciclos: Ciclo 1, Ciclo 2, Ciclo 3 y Ciclo 4.


Sources:
embeddings.md
embeddings.md


In [ ]:
query = "cual es la clave de la materia Pensamiento crítico para ingeniería y de que ciclo es?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La clave de la materia es ID0160 y pertenece al ciclo 1.


Sources:
embeddings.md
embeddings.md


In [ ]:
query = "¿cuantas materias y cuales debo aprobar antes de cargar Aprendizaje Estadistico?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Debes aprobar todas las materias básicas del primer ciclo y al menos dos materias de elección libre antes de cargar Aprendizaje Estadístico en el segundo ciclo.


Sources:
embeddings.md
embeddings.md
